# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import json

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,palasa,18.7667,84.4167,30.44,68,15,5.98,IN,1715170142
1,1,yellowknife,62.4560,-114.3525,5.79,100,100,0.89,CA,1715170142
2,2,xilin hot,43.9667,116.0333,16.31,26,100,7.64,CN,1715170142
3,3,hami,42.8000,93.4500,29.38,15,100,3.57,CN,1715170142
4,4,puerto ayora,-0.7393,-90.3518,23.08,100,100,2.68,EC,1715170143


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
map_city_humidity = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 2,
    color = "City"
)

# Display the map
map_city_humidity


:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_conditions_df = city_data_df.loc[(city_data_df["Max Temp"] <= 27) & 
                                 (city_data_df["Max Temp"] >= 21) & 
                                 (city_data_df["Cloudiness"] == 0) & 
                                 (city_data_df["Wind Speed"] < 4.5)]

# Drop any rows with null values
clean_ideal_conditions = ideal_conditions_df.dropna()

# Display sample data
clean_ideal_conditions

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
96,96,haiku-pauwela,20.9219,-156.3051,22.25,84,0,3.09,US,1715170152
153,153,varzea da palma,-17.5981,-44.7308,25.94,50,0,2.64,BR,1715170157
184,184,puerto penasco,31.3167,-113.5333,21.62,76,0,4.47,MX,1715170161
304,304,esmeraldas,-19.7625,-44.3139,24.30,71,0,3.35,BR,1715170172
313,313,al jawf,29.5000,38.7500,26.13,14,0,1.73,SA,1715170173
418,418,brandon,27.9378,-82.2859,26.07,89,0,2.06,US,1715170183
541,541,san angelo,31.4638,-100.4370,22.94,90,0,4.12,US,1715170194
547,547,tegueste,28.5167,-16.3167,25.64,22,0,4.12,ES,1715170194
553,553,mount pleasant,32.7941,-79.8626,25.59,88,0,2.57,US,1715169973


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = clean_ideal_conditions.copy(deep=True)

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
96,96,haiku-pauwela,20.9219,-156.3051,22.25,84,0,3.09,US,1715170152,
153,153,varzea da palma,-17.5981,-44.7308,25.94,50,0,2.64,BR,1715170157,
184,184,puerto penasco,31.3167,-113.5333,21.62,76,0,4.47,MX,1715170161,
304,304,esmeraldas,-19.7625,-44.3139,24.30,71,0,3.35,BR,1715170172,
313,313,al jawf,29.5000,38.7500,26.13,14,0,1.73,SA,1715170173,
418,418,brandon,27.9378,-82.2859,26.07,89,0,2.06,US,1715170183,
541,541,san angelo,31.4638,-100.4370,22.94,90,0,4.12,US,1715170194,
547,547,tegueste,28.5167,-16.3167,25.64,22,0,4.12,ES,1715170194,
553,553,mount pleasant,32.7941,-79.8626,25.59,88,0,2.57,US,1715169973,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories":"accommodation.hotel",
    "apiKey": geoapify_key,
    "limit":20
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
        
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df


Starting hotel search
haiku-pauwela - nearest hotel: Inn At Mama's Fish House
varzea da palma - nearest hotel: No hotel found
puerto penasco - nearest hotel: Hotel Paraíso Del Desierto
esmeraldas - nearest hotel: No hotel found
al jawf - nearest hotel: No hotel found
brandon - nearest hotel: Brandon Motor Lodge
san angelo - nearest hotel: Hilton Hotel
tegueste - nearest hotel: Hotel Aguere
mount pleasant - nearest hotel: Old Village Post House


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
96,96,haiku-pauwela,20.9219,-156.3051,22.25,84,0,3.09,US,1715170152,Inn At Mama's Fish House
153,153,varzea da palma,-17.5981,-44.7308,25.94,50,0,2.64,BR,1715170157,No hotel found
184,184,puerto penasco,31.3167,-113.5333,21.62,76,0,4.47,MX,1715170161,Hotel Paraíso Del Desierto
304,304,esmeraldas,-19.7625,-44.3139,24.30,71,0,3.35,BR,1715170172,No hotel found
313,313,al jawf,29.5000,38.7500,26.13,14,0,1.73,SA,1715170173,No hotel found
418,418,brandon,27.9378,-82.2859,26.07,89,0,2.06,US,1715170183,Brandon Motor Lodge
541,541,san angelo,31.4638,-100.4370,22.94,90,0,4.12,US,1715170194,Hilton Hotel
547,547,tegueste,28.5167,-16.3167,25.64,22,0,4.12,ES,1715170194,Hotel Aguere
553,553,mount pleasant,32.7941,-79.8626,25.59,88,0,2.57,US,1715169973,Old Village Post House


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
possible_destinations = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    scale = 2,
    color = "City",
    hover_cols = ["Country", "Hotel"]
)

# Display the map
possible_destinations

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Country)